# 一、文件地址管理

In [1]:
# 后期考虑先设定工作目录，再读取目录中的数据文件
news_info_train_path = './News_info_train_filter.txt'
pic_info_train_path = ''
news_pic_label_train_path = './News_pic_label_train.txt'

news_info_unlabel_path = './News_info_unlabel_filter.txt'
pic_info_unlabel_path = ''

news_info_validate_path = './News_info_validate_filter.txt'
pic_info_validate_path = ''
news_pic_label_validate_path = ''

ltp_model_path = './NLP/ltp_data_v3.4.0/cws.model'   # Ltp 3.4 分词模型库

## 二、读取原始数据

In [2]:
from instrument import read_text

news_info_train = read_text(news_info_train_path)                     # 原始新闻训练集
news_pic_label_train = read_text(news_pic_label_train_path)           # 原始新闻训练集对应的标签
news_info_validate = read_text(news_info_validate_path)               # 原始新闻验证集

# 三、通过自定义函数 data2bunch 提取每条新闻文本的id, content, pic_list, label, match_pic_list, match_text

In [3]:
from instrument import data2bunch

# 1、利用data2bunch函数对训练集与验证集的文本数据进行提取
train_bunch = data2bunch(news_info_train, label=False)            # 对训练集数据进行提取
validate_bunch = data2bunch(news_info_validate, label=False)      # 对验证集数据进行提取

# 2、利用data2bunch函数对训练集中文本和图片的标签进行提取
train_label = data2bunch(news_pic_label_train,label=True)

# 3、进行文本与标签的配对
# 判断判断文本的id与标签的id是否相同来判断文本与标签是否相匹配
for i in range(len(train_bunch.news_id)):
    if train_bunch.news_id[i] != train_label.news_pic_id[i]:
        print('出现标签不匹配！')
        print(train_bunch.news_id[i],'is not equal to',train_label.news_pic_id[i])
    else:
        pass
# （可选）4、将训练的样本与标签进行合并，合并后只有id,content and label
import pandas as pd
news_pic_train = pd.DataFrame.from_dict(train_bunch,orient='index').T
news_pic_train = news_pic_train.set_index('news_id')
# news_pic_train.head()                                        # 查看合并后的数据的前6条

In [9]:
news_pic_train.head()                                        # 查看合并后的数据的前6条
# news_pic_train.to_csv('./train.csv',encoding='utf-8')

,news_content,news_pic_list
news_id,,
D0000001,惠尔新品 | 冷色系实木多层地板系列 冷色系实木多层系列全新上市 Cool Color Mu...,P0000001.JPEG;P0000002.JPEG;P0000003.JPEG;P000...
D0000002,82周年庆 | 收到八方来贺之第一波(女士专场福利) 还有2天 就是我们中央商场的场庆啦! ...,P0000016.JPEG;P0000017.JPEG;P0000018.JPEG;P000...
D0000003,"黄河片区压轴大戏,美林康城铂悦府1月13日优惠升级 自2017年12月3日售楼部亮相以来,美...",P0000025.JPEG;P0000026.JPEG;P0000027.JPEG;P000...
D0000004,"早安 | 世界上所有的惊喜与好运,都是你积累的人品与善良。 《早安语录》第197 | 纸杯蛋...",P0000032.JPEG;P0000033.JPEG;P0000034.JPEG;P000...
D0000005,中文到底有多难?为教马克龙说这句话 翻译差点崩溃 中文到底有多难?为教马克龙说这句话 翻译差点崩溃,NULL\n


# 四、对文本进行分词

## 1、定义 text2words 函数来对句子划分为词和词组，同时：1、也将繁体中文替换为简体中文，2、将某些分错的词语进行修正

In [12]:
def text2words(need_to_segment_text, method='jieba', ltp_model_path='',postdict={}):
    '''
        该函数实现对每条新闻文本进行分词（分词方法包括jieba和ltp）,并将繁体字替换为简体字
    1、need_to_segment_text 为需要进行分词的文本列表
    2、method 为中文分词方法，可选 jieba 或 ltp
    3、ltp_model_path 为选择ltp分词方法时，ltp 分词模型在本地的目录地址
    4、postdist 数据格式：
        postdict = {'解 空间':'解空间','深度 优先':'深度优先'}
    
    '''
    from langconv import Converter
    text_words = []

    if method == 'jieba':
        import jieba
        for sentence in need_to_segment_text:
            #             print(sentence)                         # 查看新闻原文
            sentence = Converter('zh-hans').convert(sentence)     # 将繁体中文转换为简体中文
            content = sentence.replace('\r\n', '')                # 删除多余的换行符
            content = content.replace(' ', '')                    # 删除空行、多余的空格
            content_word = jieba.cut(content)
            seg_sent = ' '.join(content_word)
            for key in postdict:
                seg_sent = seg_sent.replace(key,postdict[key])    # 在分词后处理某些被分错的词和词语
            text_words.append(seg_sent)                           # 每条新闻的词与词之间通过空格隔开
    elif method == 'ltp' and ltp_model_path != '':
        from pyltp import Segmentor
        #         model_path = 'E:/Desktop/ZhuFei/Competition/NLP/ltp_data_v3.4.0/cws.model'   # Ltp 3.4 分词模型库
        segmentor = Segmentor()   # 实例化分词模块
        segmentor.load(ltp_model_path)  # 加载分词库
        for sentence in need_to_segment_text:
            #             print(sentence)                         # 查看新闻原文
            sentence = Converter('zh-hans').convert(sentence)     # 将繁体中文转换为简体中文
            content = sentence.replace('\r\n', '')                # 删除多余的换行符
            content = content.replace(' ', '')                    # 删除空行、多余的空格
            content_word = segmentor.segment(content)
            
            seg_sent = ' '.join(content_word)
            for key in postdict:
                seg_sent = seg_sent.replace(key,postdict[key])    # 在分词后处理某些被分错的词和词语
            text_words.append(seg_sent)                           # 每条新闻的词与词之间通过空格隔开
    else:
        fill_to_length = 130
        print(''.center(fill_to_length, '#'))
        print(' Method or model path is wrong! Please check it!!!! '.center(fill_to_length, '#'))
        print(''.center(fill_to_length, '#'))

    return text_words

## 2、对 text2words 函数进行测试，包括：1、模型载入；2、繁体字替换；3、错分词处理

In [13]:
ltp_model_path = './NLP/ltp_data_v3.4.0/cws.model'   # Ltp 3.4 分词模型库
print(train_bunch.news_content[71])
postdict = {'男友 好友':'男友好友','久坐 不 动':'久坐不动'}
text2words(train_bunch.news_content[68:71],method='jieba',ltp_model_path=ltp_model_path) # ltp 分词

沈剛《漢代國家統治方式研究》出版(附:前言) 作者: 沈剛 出版社: 社會科學文獻出版社 副標題: 列卿、宗室、信仰與基層社會 出版年:2017-12-1 頁數: 280 定價: 89 ISBN:9787520118149 內容簡介:本書從列卿、宗室、民間信仰和地方社會管理四個方面對漢代國家統治方式進行了探討。首先,從漢代列卿的內涵、淵源、選任以及對廷尉等卿職的個案考察來分析中央職官的設置特點;其次,從兩漢國家對諸侯王和普通宗室群體優撫與控制措施及宗正的職能來觀察漢代宗室管理問題;再次,通過民間信仰的特徵、與地方行政機構的互動,以及東漢皇權與讖緯的聯繫來探究漢代信仰與政治間的關係;最後,漢代國家對居住區形態、民間秩序、社會事務的干預也顯示了這一時期國家權力為控制社會所做出的努力。 作者簡介:沈剛,男,出生於1973年,遼寧寬甸人,吉林大學古籍研究所教授,歷史學博士。主要從事秦漢魏晉史、簡牘學教學與研究。已出版《秦漢時期的客階層研究》《居延漢簡語詞匯釋》兩部著作,獨撰並發表專業論文50餘篇,主持國家社科基金,教育部、全國高校古委會、吉林省社科基金,以及吉林大學等各級各類項目10余項,另參與各級各類項目多項。 目錄 前 言 第一編 漢代的列卿 第一章 漢代“九卿”的內涵 一 何為“九卿”:學術史回顧 二 與“卿”相關語詞的分析 三 九卿的內涵 第二章 秦代列卿問題發微 一 秦代的卿職 二 秦卿溯源 三 秦代列卿的原始性 第三章 漢代的廷尉 一 廷尉的主要職掌 二 廷尉權力的局限與制衡因素 三 廷尉的遷選特點及其僚屬 第四章 漢代中尉的幾個問題 一 漢代中尉的淵源 二 中尉的職責與演變 三 有關中尉軍事職能的幾個問題 第五章 漢代的太僕 一 太僕的性質與地位 二 太僕的主要職掌 三 太僕的遷轉和來源 第六章 漢代列卿選任中的非制度化現象舉隅 一 世官現象及其成因 二 政策導向所施加的影響 三 影響列卿選任的其他非制度化因素 附論一 漢代中央的廄 一 漢代中央廄的設置與分佈 二 廄馬的性質 三 中央廄馬的來源與使用 四 中央諸廄的職官設置 附論二 戰國秦漢時期的舍人 一 戰國時代的舍人 二 秦與秦漢之際舍人性質的轉化 三 漢代的舍人 第二編 漢代宗室及其管理 第七章 西漢宗室管理問題 一 中央政權對宗室諸王的控制手段 二 中央對諸侯王犯罪活動的懲處 三 中央政權對一般宗室的

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zf\AppData\Local\Temp\4\jieba.cache
Loading model cost 1.191 seconds.
Prefix dict has been built succesfully.


['跑步 伤 膝盖 ? 久坐 不 动 膝盖 更糟 跑步 是 一种 最 常见 的 运动 方式 。 众所周知 , 跑步 能 锻炼 心肺 功能 , 增强 肌肉 耐力 , 持续 有效 的 慢跑 还 能 起到 消脂 瘦身 的 作用 。 在 如今 全民 健身 的 浪潮 之下 , 跑步 由于 运动 条件 简单 , 不 受 场地 限制 , 运动 方式 自由 等 特征 从而 拥有 了 广泛 的 群众 基础 。 由于 跑步 运动 基础 如此 广泛 , 因此 也 不可避免 地 导致 许多 没有 经过 相关 知识 学习 , 进行 持续 适应性 训练 的 跑友 盲目 参与 。 虽然 跑步 好处多多 , 但 若 对 相关 知识 不够 了解 , 姿势 不 标准 , 或者 适应性 训练 不 到位 等等 , 这样 不仅 起 不到 健身 塑形 的 效果 , 甚至 还会 给 身体 带来 损害 。 所以 为什么 许多 热爱 跑步 , 积极 分享 跑步 乐趣 的 跑友 最终 淡出 了 跑步 圈 , 答案 也 很 简单 , 跑步 损伤 。 其实 以 跑步 膝 尤甚 。 什么 是 跑步 膝 ? 即 髌 股 疼痛 综合征 。 很多 人 了解 到 跑步 的 这 一面 后 会 犹豫 到底 要 不要 跑步 。 你 究竟 该不该 跑 呢 , 让 医学 权威 来 告诉 你 。 美国 权威 期刊 《 骨科 与 运动 物理 治疗 杂志 》 ( JOSPT ® ) 6 月刊 的 杂志 发表 了 一项 最新 研究 : 业 馀 跑步 爱好者 与 久坐 不动 的 群体 和 竞争性 跑步 者 ( 职业 运动员 或 高强度 训练 者 ) 相比 , 有 更 低 的 可能性 患有 膝关节 和 髋关节 骨性 关节炎 。 西班牙 , 瑞典 , 美国 和 加拿大 的 国际 研究 团队 旨在 评估 髋关节 和 膝关节 骨关节炎 与 跑步 的 关系 , 并 探讨 跑步 强度 和 年数 对 该 关系 的 影响 。 他们 审查 了 25 项 研究 , 其中 包括 125 , 810 人 , 最终 选出 17 项 研究 , 共 涉及 114 , 829 人 作为 分析 对象 。 如果 跑步 者 为 专业 / 精英 运动员 或 参加 国际 比赛 , 其 被 认为 是 “ 竞争性 跑步 者 ” 。 业 馀 爱好者 则 是 那些 跑步 不 专业 , 只是 在业 馀

## 3、对训练集和验证集中的新闻文本进行分词（包括jieba分词与ltp分词）

In [14]:
ltp_model_path = './NLP/ltp_data_v3.4.0/cws.model'   # Ltp 3.4 分词模型库

# 对训练集中的新闻文本进行分词 
train_content_words_jieba = text2words(train_bunch.news_content,method='jieba')    # jieba 分词
train_content_words_ltp = text2words(train_bunch.news_content,method='ltp',ltp_model_path=ltp_model_path) # ltp 分词

# 对验证集中的新闻文本进行分词 
validate_content_words_jieba = text2words(validate_bunch.news_content,method='jieba')   # jieba 分词 
validate_content_words_ltp = text2words(validate_bunch.news_content,method='ltp',ltp_model_path=ltp_model_path) # ltp 分词

## 4、将标签为'1'的样本再复制一份

In [15]:
### 考虑到 先扩充样本再分词 会比 先分词再扩充样本 需要更长的时间，所以采取先分词再扩充样本的方案
# 按标签将数据分离开来
label_0_id = []               # 用以获取标签为 0 的数据 ID
label_1_id = []               # 用以获取标签为 1 的数据 ID
label_2_id = []               # 用以获取标签为 2 的数据 ID

for index in range(len(train_label.news_pic_id)):
    label = train_label.news_pic_label[index]
    if label == '0':
        label_0_id.append(train_label.news_pic_id[index])
    elif label == '1':
        label_1_id.append(train_label.news_pic_id[index])
    elif label == '2':
        label_2_id.append(train_label.news_pic_id[index])
    else:
        print("ID为 %s 的样本标签不为‘0’，‘1’，‘2’中的任何一个！！" % train_label.news_pic_id[index])

print('The length of label_0_id is %d' % len(label_0_id))
print('The length of label_1_id is %d' % len(label_1_id))
print('The length of label_2_id is %d' % len(label_2_id))

# 由于类别之间存在样本不均衡问题，将标签为1的样本复制一份，将其样本量扩大一倍（以标签为0的样本为基底，进行随机插入）
import random
import copy
new_id = copy.deepcopy(label_0_id)   # 深复制

print("由标签为'0'的样本组成的训练集中共含数据 %d 条！" % len(new_id))
for i in label_1_id:         # 将标签为'1'的样本随机插入标签为'0'的样本中，将标签为'0'和'1'的样本合并
    len_new_id = len(new_id)
    insert_index = random.sample(range(len_new_id),1)[0]
    new_id.insert(insert_index,i)
print("由标签为'0'和'1'的样本组成的训练集中共含数据 %d 条！" % len(new_id))
for j in label_1_id:         # 将标签为'1'的样本插入合并的数据集中，将标签为'1'的样本量扩大一倍
    len_new_id = len(new_id)
    insert_index = random.sample(range(len_new_id),1)[0]
    new_id.insert(insert_index,j)
print("将标签'1'的样本复制一份后，训练集中共含数据 %d 条！" % len(new_id))
for k in label_2_id:         # 将标签为'2'的样本插入上述数据中，组合成新的训练集
    len_new_id = len(new_id)
    insert_index = random.sample(range(len_new_id),1)[0]
    new_id.insert(insert_index,k)
print("再将标签为'2'的样本放入，此时，训练集中共 %d 条数据！" % len(new_id))


The length of label_0_id is 23899
The length of label_1_id is 7533
The length of label_2_id is 17048
由标签为'0'的样本组成的训练集中共含数据 23899 条！
由标签为'0'和'1'的样本组成的训练集中共含数据 31432 条！
将标签'1'的样本复制一份后，训练集中共含数据 38965 条！
再将标签为'2'的样本放入，此时，训练集中共 56013 条数据！


## 5、将用jieba方法和ltp方法分词后的数据进行 Bunch 化，方便以后的调用

In [16]:
from sklearn.datasets.base import Bunch

# 1、将训练集（含标签及和标签有关的配图列表和文本）保存为一个本地的 Bunch 对象
TRAIN_BUNCH = Bunch(news_id=[],news_words_jieba=[],news_words_ltp=[],news_pic_list=[],news_pic_label=[],news_pic_pic=[],news_pic_text=[])
# 基于分词后的结果进行类别不平衡的处理（标签为'1'的样本复制一份）
for ID in new_id:
    data_index = train_label.news_pic_id.index(ID)
    TRAIN_BUNCH.news_id.append(ID)
#     TRAIN_BUNCH.news_content = train_bunch.news_content
    TRAIN_BUNCH.news_words_jieba.append(train_content_words_jieba[data_index]) 
    TRAIN_BUNCH.news_words_ltp.append(train_content_words_ltp[data_index])
    TRAIN_BUNCH.news_pic_list.append(train_bunch.news_pic_list[data_index])
    TRAIN_BUNCH.news_pic_label.append(train_label.news_pic_label[data_index])
    TRAIN_BUNCH.news_pic_pic.append(train_label.news_pic_pic[data_index])
    TRAIN_BUNCH.news_pic_text.append(train_label.news_pic_text[data_index])

# TRAIN_BUNCH.news_id = train_bunch.news_id
# # TRAIN_BUNCH.news_content = train_bunch.news_content
# TRAIN_BUNCH.news_words_jieba = train_content_words_jieba
# TRAIN_BUNCH.news_words_ltp = train_content_words_ltp
# TRAIN_BUNCH.news_pic_list = train_bunch.news_pic_list
# TRAIN_BUNCH.news_pic_label = train_label.news_pic_label
# TRAIN_BUNCH.news_pic_pic = train_label.news_pic_pic
# TRAIN_BUNCH.news_pic_text = train_label.news_pic_text

# 2、将验证集（contain id, content and pictures list）保存为一个本地的 Bunch 对象
VALIDATE_BUNCH = Bunch(news_id=[],news_words_jieba=[],news_words_ltp=[],news_pic_list=[])
VALIDATE_BUNCH.news_id = validate_bunch.news_id
VALIDATE_BUNCH.news_words_jieba = validate_content_words_jieba
VALIDATE_BUNCH.news_words_ltp = validate_content_words_ltp
VALIDATE_BUNCH.news_pic_list = validate_bunch.news_pic_list

In [26]:
len(TRAIN_BUNCH.news_id)

56013

## 6、由于ltp分词比较耗时间，考虑将用jieba方法和ltp方法分词后的训练集和验证集数据以 Bunch 对象保存至本地指定的目录

In [17]:
from instrument import save_bunch

train_bunch_path = './data_bunch/train_bunch_balance.dat'       # 训练集保存地址，由于后期需要调用该数据且地址也相同，故选择该地址
save_bunch(train_bunch_path,TRAIN_BUNCH)

validate_bunch_path = './data_bunch/validate_bunch.dat' # 验证集保存地址，由于后期需要调用该数据且地址也相同，故选择该地址
save_bunch(validate_bunch_path,VALIDATE_BUNCH)

In [1]:
from instrument import read_bunch

train_bunch_path = './data_bunch/train_bunch_balance.dat'       # 训练集保存地址，由于后期需要调用该数据且地址也相同，故选择该地址
train_bunch = read_bunch(train_bunch_path)


In [2]:
dir(train_bunch)

['news_id',
 'news_pic_label',
 'news_pic_list',
 'news_pic_pic',
 'news_pic_text',
 'news_words_jieba',
 'news_words_ltp']

In [4]:
for index in range(10): #len(train_bunch.news_id)
    print(train_bunch.news_id[index])
    print(train_bunch.news_pic_list[index])
    print(train_bunch.news_words_jieba[index])
    print(train_bunch.news_pic_label[index])
    

D0000001
P0000001.JPEG;P0000002.JPEG;P0000003.JPEG;P0000004.JPEG;P0000005.JPEG;P0000006.JPEG;P0000007.JPEG;P0000008.JPEG;P0000009.JPEG;P0000010.JPEG;P0000011.JPEG;P0000012.JPEG;P0000013.JPEG;P0000014.JPEG;P0000015.JPEG

惠尔 新品 | 冷色 系 实木 多层 地板 系列 冷色 系 实木 多层 系列 全新 上市 CoolColorMultiplyWoodfloorWELLFLOOR 在 很多 人 心中 , 冰冷 与 阴暗 都 是 冷色 系 家居设计 的 标签 , 因此 , 人们 在 装修 的 时候 常常 会 把 它 拒之门外 。 不过 近几年来 , 冷色 系 再次 回归 到 家装 界 , 也 受到 了 越来越 多 小资 人士 的 欢迎 。 冷色 系 家居 之所以 能 成功 逆袭 , 在于 它 独特 的 魅力 。 冷色 系给 人 一种 安静 、 沉稳 、 踏实 的 感觉 , 能够 营造 一个 宁静 安详 的 家居 氛围 , 让 人们 在 喧闹 的 都市生活 中 回归 到 安逸 愉悦 的 氛围 中 。 尤其 是 在 情绪 火爆 或 亢奋 紧张 的 时候 , 置身 在 冷色 系 的 色彩 氛围 里 , 人们 便会 冷静下来 , 理智 下来 , 让 身体 、 大脑 和 心灵 一同 得到 休息 和 放松 。 2017 年 , 惠尔 冷色 系 多层 实 木地板 全新 上市 ! 惠尔 2017 推出 的 冷色 系 多层 实 木地板 系列 , 由 天然 桦 木材 加工 制成 , 纹理 直且 明显 , 材质 结构 细腻 而 柔和 光滑 , 质地 较软 或 适中 。 在 木材 天然 纹理 上 采用 独特 的 全手工 擦色 及 精细 拉丝 工艺 , 使得 每 一块 地板 呈现 完美 质感 纹理 。 此次 推出 的 新品 , 采用 无醛 芯 E0 环保 基材 , 采用 生物 大豆胶 替代 传统 的 含 甲醛 胶合剂 , 让 居室 时尚 精致 的 同时 , 更加 的 环保 健康 。 表层 涂有 除醛 UV 油漆 , 更进一步 防醛 控醛 , 且 漆面 渗透 好 , 防腐 耐磨 ,

# 五、创建分类模型

## （一）、贝叶斯模型

## 对 jieba 和 ltp 分词的结果用 search_best_para_bayes 函数在预置的参数空间内寻找使贝叶斯模型在测试集上表现最佳的参数组合，并基于两种方法分别进行预测

In [1]:
# 各参数的候选集
min_df_list = [0, 0.01, 0.001]               # 尽量小
max_df_list = [0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.999]   # 尽量大
feature_percent_list = [0.8,0.85,0.9,0.95]                 # 大于 0.5
alpha_list = [0.5,0.09,0.099,0.001]             # 尽量小

### 1、jieba

In [2]:
# 1.1、基于 jieba 分词结果寻找最佳的参数组合
from instrument import search_best_para_bayes

best_min_df_jieba, best_max_df_jieba, best_feature_percentage_jieba, best_alpha_jieba,_ ,_=search_best_para_bayes(min_df_list,
                                                                                                                  max_df_list,
                                                                                                                  feature_percent_list,
                                                                                                                  alpha_list,method = 'jieba')

**********************************************************************************************************************************
Accuracy is 0.607772
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6077719590573673
             precision    recall  f1-score   support

          0       0.56      0.82      0.67      7150
          1       0.61      0.20      0.30      4533
          2       0.70      0.67      0.68      5121

avg / total       0.62      0.61      0.57     16804

Current best parameters and result : [0, 0.6, 0.7, 1e-05, 0.626208268509781, 0.6077719590573673]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.612176
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.612175672458938

**********************************************************************************************************************************
Accuracy is 0.614794
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6147940966436563
             precision    recall  f1-score   support

          0       0.58      0.80      0.67      7150
          1       0.60      0.25      0.36      4533
          2       0.69      0.68      0.69      5121

avg / total       0.62      0.61      0.59     16804

Current best parameters and result : [0, 0.7, 1, 1e-05, 0.6445203907266189, 0.6147940966436563]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.607772
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6077719590573673

**********************************************************************************************************************************
Accuracy is 0.614199
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6141990002380385
             precision    recall  f1-score   support

          0       0.58      0.80      0.67      7150
          1       0.61      0.24      0.35      4533
          2       0.69      0.68      0.68      5121

avg / total       0.62      0.61      0.59     16804

Current best parameters and result : [0, 0.85, 0.9, 1e-05, 0.6383483383916958, 0.6141990002380385]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.614794
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6147940966436

**********************************************************************************************************************************
Accuracy is 0.440371
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.44037134015710544
             precision    recall  f1-score   support

          0       0.43      1.00      0.60      7150
          1       0.25      0.00      0.00      4533
          2       0.91      0.05      0.10      5121

avg / total       0.53      0.44      0.29     16804

Current best parameters and result : [0.01, 0.6, 0.8, 0.001, 0.44270448111402994, 0.44037134015710544]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.442811
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.44281123

**********************************************************************************************************************************
Accuracy is 0.439836
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4398357533920495
             precision    recall  f1-score   support

          0       0.43      1.00      0.60      7150
          1       0.25      0.00      0.00      4533
          2       0.90      0.05      0.09      5121

avg / total       0.53      0.44      0.29     16804

Current best parameters and result : [0.01, 0.75, 0.7, 0.01, 0.44201586370476165, 0.4398357533920495]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.440371
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4403713401

...

**********************************************************************************************************************************
Accuracy is 0.505534
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.5055343965722447
             precision    recall  f1-score   support

          0       0.47      0.97      0.63      7150
          1       0.51      0.00      0.01      4533
          2       0.81      0.30      0.44      5121

avg / total       0.58      0.51      0.40     16804

Current best parameters and result : [0.001, 0.7, 1, 0.001, 0.508939274146242, 0.5055343965722447]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.484885
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4848845512973

**********************************************************************************************************************************
Accuracy is 0.498096
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4980956915020233
             precision    recall  f1-score   support

          0       0.46      0.97      0.63      7150
          1       0.50      0.00      0.01      4533
          2       0.82      0.27      0.41      5121

avg / total       0.58      0.50      0.39     16804

Current best parameters and result : [0.001, 0.85, 0.9, 0.001, 0.502104108750542, 0.4980956915020233]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.505534
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.5055343965

**********************************************************************************************************************************
Accuracy is 0.448108
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4481075934301357
             precision    recall  f1-score   support

          0       0.44      1.00      0.61      7150
          1       0.62      0.01      0.01      4533
          2       0.89      0.08      0.14      5121

avg / total       0.62      0.45      0.30     16804

Current best parameters and result : [0.005, 0.6, 0.8, 0.01, 0.45055982044938664, 0.4481075934301357]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.451083
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4510830754

**********************************************************************************************************************************
Accuracy is 0.444835
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.44483456319923825
             precision    recall  f1-score   support

          0       0.43      1.00      0.61      7150
          1       0.66      0.01      0.01      4533
          2       0.91      0.06      0.12      5121

avg / total       0.64      0.44      0.30     16804

Current best parameters and result : [0.005, 0.75, 0.7, 0.001, 0.4467851768726568, 0.44483456319923825]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.448108
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4481075

**********************************************************************************************************************************
Accuracy is 0.452333
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4523327779100214
             precision    recall  f1-score   support

          0       0.44      0.99      0.61      7150
          1       0.62      0.01      0.01      4533
          2       0.88      0.09      0.17      5121

avg / total       0.62      0.45      0.31     16804

Current best parameters and result : [0.005, 0.85, 1, 0.001, 0.45489555969292766, 0.4523327779100214]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.444835
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4448345631

In [3]:
# 1.2、用搜索到的参数组合对验证集进行预测
from instrument import read_bunch, perdict_bayes

validate_bunch_path = './data_bunch/validate_bunch.dat'
validate_bunch = read_bunch(validate_bunch_path)
# 对验证集进行预测
validate_predict_jieba = perdict_bayes(best_min_df_jieba,best_max_df_jieba,best_feature_percentage_jieba,best_alpha_jieba,method='jieba')

# 整合验证集预测结果的提交文件
bayes_pic_id = []    # 验证集对应的配图 ID 列表
bayes_text = []      # 验证集对应的文本片段列
for i in range(len(validate_bunch.news_id)):
    bayes_pic_id.append('NULL')
    bayes_text.append('NULL')

bayes_result_jieba = []    # 验证集的预测结果
for i in range(len(validate_bunch.news_id)):
    bayes_result_jieba.append(validate_bunch.news_id[i]+'\t'+validate_predict_jieba[i]+'\t'+bayes_pic_id[i]+'\t'+bayes_text[i])

In [4]:
# 1.3保存预测结果用以提交
from instrument import save_text
save_path_jieba = './submittion/submittion_validate_jieba.txt'
save_text(save_path_jieba,bayes_result_jieba)

### 2、ltp

In [5]:
# 2.1、基于 ltp 分词结果寻找最佳的参数组合
from instrument import search_best_para_bayes
best_min_df_ltp, best_max_df_ltp, best_feature_percentage_ltp, best_alpha_ltp,_ ,_=search_best_para_bayes(min_df_list,
                                                                                                          max_df_list,
                                                                                                          feature_percent_list,
                                                                                                          alpha_list,method = 'ltp')

**********************************************************************************************************************************
Accuracy is 0.613663
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6136634134729826
             precision    recall  f1-score   support

          0       0.58      0.80      0.67      7150
          1       0.59      0.24      0.34      4533
          2       0.68      0.69      0.68      5121

avg / total       0.62      0.61      0.59     16804

Current best parameters and result : [0, 0.6, 0.7, 1e-05, 0.6354663470121655, 0.6136634134729826]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.614378
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.61437752915972

**********************************************************************************************************************************
Accuracy is 0.618127
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6181266365151155
             precision    recall  f1-score   support

          0       0.59      0.77      0.67      7150
          1       0.59      0.29      0.39      4533
          2       0.67      0.69      0.68      5121

avg / total       0.62      0.62      0.60     16804

Current best parameters and result : [0, 0.7, 1, 1e-05, 0.6515340865617588, 0.6181266365151155]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.613663
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6136634134729826

**********************************************************************************************************************************
Accuracy is 0.616817
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6168174244227564
             precision    recall  f1-score   support

          0       0.59      0.78      0.67      7150
          1       0.59      0.27      0.38      4533
          2       0.68      0.69      0.68      5121

avg / total       0.62      0.62      0.60     16804

Current best parameters and result : [0, 0.85, 0.9, 1e-05, 0.6465097299089495, 0.6168174244227564]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.618127
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.6181266365151

e:\software\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**********************************************************************************************************************************
Accuracy is 0.437634
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.43763389669126396
             precision    recall  f1-score   support

          0       0.43      1.00      0.60      7150
          1       0.11      0.00      0.00      4533
          2       0.92      0.04      0.08      5121

avg / total       0.49      0.44      0.28     16804

Current best parameters and result : [0.01, 0.6, 0.8, 0.001, 0.4401285419163967, 0.43763389669126396]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.438169
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.438169483

...

**********************************************************************************************************************************
Accuracy is 0.497322
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4973220661747203
             precision    recall  f1-score   support

          0       0.46      0.97      0.63      7150
          1       0.55      0.01      0.01      4533
          2       0.82      0.27      0.41      5121

avg / total       0.59      0.50      0.39     16804

Current best parameters and result : [0.001, 0.7, 1, 1e-05, 0.5009819174169196, 0.4973220661747203]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.472745
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.472744584622

**********************************************************************************************************************************
Accuracy is 0.491728
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4917281599619138
             precision    recall  f1-score   support

          0       0.46      0.98      0.62      7150
          1       0.49      0.00      0.01      4533
          2       0.83      0.25      0.38      5121

avg / total       0.58      0.49      0.38     16804

Current best parameters and result : [0.001, 0.85, 0.9, 0.001, 0.49463133464255654, 0.4917281599619138]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.497322
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.49732206

**********************************************************************************************************************************
Accuracy is 0.444835
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.44483456319923825
             precision    recall  f1-score   support

          0       0.43      1.00      0.60      7150
          1       0.60      0.01      0.01      4533
          2       0.92      0.06      0.12      5121

avg / total       0.63      0.44      0.30     16804

Current best parameters and result : [0.005, 0.6, 0.8, 0.01, 0.44744828993343366, 0.44483456319923825]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.447572
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.44757200

**********************************************************************************************************************************
Accuracy is 0.443942
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4439419185908117
             precision    recall  f1-score   support

          0       0.43      1.00      0.60      7150
          1       0.55      0.00      0.01      4533
          2       0.93      0.06      0.12      5121

avg / total       0.61      0.44      0.29     16804

Current best parameters and result : [0.005, 0.75, 0.7, 0.001, 0.4468106812211482, 0.4439419185908117]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.444835
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.444834563

**********************************************************************************************************************************
Accuracy is 0.447929
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.4479290645084504
             precision    recall  f1-score   support

          0       0.44      1.00      0.61      7150
          1       0.62      0.01      0.01      4533
          2       0.90      0.07      0.14      5121

avg / total       0.63      0.45      0.30     16804

Current best parameters and result : [0.005, 0.85, 1, 0.01, 0.45101889872223216, 0.4479290645084504]
**********************************************************************************************************************************
**********************************************************************************************************************************
Accuracy is 0.443942
The accuracy of Naive_Bayes Classifier on dataset wich splis from train data is : 0.44394191859

In [6]:
# 2.2、用搜索到的参数组合对验证集进行预测
from instrument import read_bunch, perdict_bayes
validate_bunch_path = './data_bunch/validate_bunch.dat'
validate_bunch = read_bunch(validate_bunch_path)

# 对验证集进行预测
validate_predict_ltp = perdict_bayes(best_min_df_ltp,best_max_df_ltp,best_feature_percentage_ltp,best_alpha_ltp,method='ltp')

# 整合验证集预测结果的提交文件
bayes_pic_id = []    # 验证集对应的配图 ID 列表
bayes_text = []      # 验证集对应的文本片段列
for i in range(len(validate_bunch.news_id)):
    bayes_pic_id.append('NULL')
    bayes_text.append('NULL')

bayes_result_ltp = []    # 验证集的预测结果
for i in range(len(validate_bunch.news_id)):
    bayes_result_ltp.append(validate_bunch.news_id[i]+'\t'+validate_predict_ltp[i]+'\t'+bayes_pic_id[i]+'\t'+bayes_text[i])

In [7]:
# 2.3保存预测结果用以提交
from instrument import save_text
save_path_ltp = './submittion/submittion_validate_ltp.txt'
save_text(save_path_ltp,bayes_result_ltp)